# Start 

Below is an example of what a user would have to create in order to get the LShaped.jl package to run their own custom scenario-based model. I use, for simplicity, an example out of Introduction to Stochastic Programming (2nd). This comes from Ch. 5, where the L-Shaped Method is introduced.

In [1]:
using Pkg
Pkg.activate("..")

In [10]:
Pkg.add("OrderedCollections")

In [11]:
using JuMP
using Gurobi
using CSV
using DataFrames
using OrderedCollections

In [3]:
using LShaped

┌ Info: Precompiling LShaped [9bb95bf9-7f86-40d5-bb05-f9cfaa3d19d3]
└ @ Base loading.jl:1317


### Create subproblem generating function (dependent on scenario id)

In [4]:
function create_model_var_obj(dfvar)
    
    model = Model(with_optimizer(Gurobi.Optimizer; OutputFlag = 0))

    cexpr = JuMP.GenericAffExpr(0.0, OrderedDict{VariableRef,Float64}())
    for i = 1:size(dfvar)[1]
        
        if i % 1000 == 0
            println("Creating variable $(i)...")
        end

        vname31 = dfvar[i,2]
        vlb = dfvar[i,3]
        vub = dfvar[i,4]
        cost = dfvar[i,5]
        
	#attempt to change vname::String31 to String
	vname = ""
	for s = 1:length(vname31)
	    vname = string(vname,vname31[s])
	end

	#println("$(vname), $(typeof(vname))")

        if vlb > -Inf
            if vub < Inf
                vref = @variable(model, base_name = vname, lower_bound = vlb, upper_bound = vub)
            else
                vref = @variable(model, base_name = vname, lower_bound = vlb)
            end
        else
            if vub < Inf
                vref = @variable(model, base_name = vname, upper_bound = vub)
            else
                vref = @variable(model, base_name = string(vname))
            end
        end

        add_to_expression!(cexpr, cost, vref)

    end

    @objective(model, Min, cexpr)
    
    return model
    
end

function add_model_cons!(model, dfvar, dfcon, dfmat)
    
    #println("Creating Sparse Matrix...")
    #con_mat = sparse(dfmat[:,1], dfmat[:,2], dfmat[:,3])
    
    count = 1
    
    for connum = 1:size(dfcon)[1]
        
        if connum % 1000 == 0
            println("Creating Constraint $(connum)...")
        end
        
        conexpr = JuMP.GenericAffExpr(0.0, OrderedDict{VariableRef,Float64}())
        
        varids = Int64[];
        vals = Float64[];
                
        t1 = time_ns()
        for i = count:size(dfmat,1)

            # temporary. probably should do something like "find nonzeros for this row"
            #(varids, vals) = findnz(con_mat[rownum,:])
            
            if connum == dfmat[i,1]
                push!(varids, dfmat[i,2])
                push!(vals, dfmat[i,3])
            else
                count = i
                break
            end
            
        end
        #=
        if connum < 20 || connum == size(dfcon)[1]
            println("varids = $(varids)")
            println("vals = $(vals)")
            println("count = $(count)")
        end
        =#
        t2 = time_ns()
            
            
        for  i = 1:size(varids,1)

            coeff = vals[i] #dfcon[rownum,colnum]

	    vname_old = dfvar[varids[i],2]
	    vname = ""
	    for s = 1:length(vname_old)
		vname = string(vname,vname_old[s])
	    end

            vref = variable_by_name(model, vname)

            add_to_expression!(conexpr, coeff, vref)

        end
        t3 = time_ns()

        contype = dfcon[connum, 3]

        if occursin("EqualTo", contype)

            value = dfcon[connum,4]

            @constraint(model, conexpr == value)

        elseif occursin("GreaterThan", contype)

            value = dfcon[connum,4]

            @constraint(model, conexpr >= value)

         elseif occursin("LessThan", contype)

            value = dfcon[connum,4]

            @constraint(model, conexpr <= value)

         elseif occursin("Interval", contype)

            lb = dfcon[connum,4]
            ub = dfcon[connum,5]

            @constraint(model, lb <= conexpr <= ub)

         else 

            println("constraint missing from model. This is not good.")

        end
            t4 = time_ns()

            if connum % 1000 == 0
                nztime = (t2-t1)/1e9
                exptime = (t3-t2)/1e9
                contime = (t4-t3)/1e9
                tottime = (t4-t1)/1e9
                println("For this iteration: nzt =$(nztime), expt = $(exptime), cont = $(contime), tot = $(tottime)")
            end
        end
    
    return
    
end


add_model_cons! (generic function with 1 method)

In [5]:
# creates the model whose info is stored in "dir/scenid"
function subproblem_uploader(scenid)
    
    dir = "../../ScenarioPrimal/ScenarioPrimal/jumpmodels/wsgep/twostage/ts288/"
    
    varfile = string(dir, scenid, "/vars_eff.csv")
    confile = string(dir, scenid, "/cons_eff.csv")
    matfile = string(dir, scenid, "/mat_eff.csv")

    println("Loading in variable dataframe...")
    dfvar = DataFrame(CSV.File(varfile));
    println("Loading in constraint dataframe...")
    dfcon = DataFrame(CSV.File(confile));
    println("Loading in constraint matrix...")
    dfmat = DataFrame(CSV.File(matfile));
    
    println("Creating model...")
    model = create_model_var_obj(dfvar);
    
    add_model_cons!(model, dfvar, dfcon, dfmat)
    
    return model
    
end


subproblem_uploader (generic function with 1 method)

### Create variable dictionary. Key 1 maps to stage 1 variables. Key 2 maps to stage 2 variables.

TODO: have it construct this inside of the package (e.g. only give instructions to do it)

In [6]:
dir = "../../ScenarioPrimal/ScenarioPrimal/jumpmodels/wsgep/twostage/ts288/"

scenid = 1

varfile = string(dir, scenid, "/vars_eff.csv")
#confile = string(dir, scenid, "/cons.csv")

dfvar = DataFrame(CSV.File(varfile));
#dfcon = DataFrame(CSV.File(confile));

dfvar[size(dfvar,1),1]

x_init = ones(150);

function variable_dict_primal(varlist, x)
    vdict = Dict{Int,Any}()
    vdict[1]=[]
    vdict[2]=[]
    i = 0
    for name in varlist
        if occursin("m_EE", name) || occursin("r_WE", name)
            
            i+=1
            
            # for some reason, the lower bound on these variables is not showing...I'm not sure why.
            lb = 0
            ub = Inf
            #if has_lower_bound(var) == 1
            #    lb = lower_bound(var)
            #end
            #if has_upper_bound(var) == 1
            #    ub = upper_bound(var)
            #end
            push!(vdict[1], (name, lb, ub, x[i]))
        else
            push!(vdict[2], name)
        end
    end
    return vdict
end

v_dict = variable_dict_primal(dfvar[:,2], x_init);

In [7]:
v_dict[1]

150-element Vector{Any}:
 ("r_WE[101_WIND_2]", 0, Inf, 1.0)
 ("r_WE[102_WIND_2]", 0, Inf, 1.0)
 ("r_WE[103_WIND_2]", 0, Inf, 1.0)
 ("r_WE[104_WIND_2]", 0, Inf, 1.0)
 ("r_WE[105_WIND_2]", 0, Inf, 1.0)
 ("r_WE[106_WIND_2]", 0, Inf, 1.0)
 ("r_WE[107_WIND_2]", 0, Inf, 1.0)
 ("r_WE[108_WIND_2]", 0, Inf, 1.0)
 ("r_WE[109_WIND_2]", 0, Inf, 1.0)
 ("r_WE[110_WIND_2]", 0, Inf, 1.0)
 ("r_WE[111_WIND_2]", 0, Inf, 1.0)
 ("r_WE[112_WIND_2]", 0, Inf, 1.0)
 ("r_WE[113_WIND_2]", 0, Inf, 1.0)
 ⋮
 ("m_EE[314_STORAGE_1]", 0, Inf, 1.0)
 ("m_EE[315_STORAGE_1]", 0, Inf, 1.0)
 ("m_EE[316_STORAGE_1]", 0, Inf, 1.0)
 ("m_EE[317_STORAGE_1]", 0, Inf, 1.0)
 ("m_EE[318_STORAGE_1]", 0, Inf, 1.0)
 ("m_EE[319_STORAGE_1]", 0, Inf, 1.0)
 ("m_EE[320_STORAGE_1]", 0, Inf, 1.0)
 ("m_EE[321_STORAGE_1]", 0, Inf, 1.0)
 ("m_EE[322_STORAGE_1]", 0, Inf, 1.0)
 ("m_EE[323_STORAGE_1]", 0, Inf, 1.0)
 ("m_EE[324_STORAGE_1]", 0, Inf, 1.0)
 ("m_EE[325_STORAGE_1]", 0, Inf, 1.0)

## create first stage problem

In [8]:
function create_first_stage()
    
    names = ["r_WE[101_WIND_2]"
"r_WE[102_WIND_2]"
"r_WE[103_WIND_2]"
"r_WE[104_WIND_2]"
"r_WE[105_WIND_2]"
"r_WE[106_WIND_2]"
"r_WE[107_WIND_2]"
"r_WE[108_WIND_2]"
"r_WE[109_WIND_2]"
"r_WE[110_WIND_2]"
"r_WE[111_WIND_2]"
"r_WE[112_WIND_2]"
"r_WE[113_WIND_2]"
"r_WE[114_WIND_2]"
"r_WE[115_WIND_2]"
"r_WE[116_WIND_2]"
"r_WE[117_WIND_2]"
"r_WE[118_WIND_2]"
"r_WE[119_WIND_2]"
"r_WE[120_WIND_2]"
"r_WE[121_WIND_2]"
"r_WE[122_WIND_1]"
"r_WE[122_WIND_2]"
"r_WE[123_WIND_2]"
"r_WE[124_WIND_2]"
"r_WE[201_WIND_2]"
"r_WE[202_WIND_2]"
"r_WE[203_WIND_2]"
"r_WE[204_WIND_2]"
"r_WE[205_WIND_2]"
"r_WE[206_WIND_2]"
"r_WE[207_WIND_2]"
"r_WE[208_WIND_2]"
"r_WE[209_WIND_2]"
"r_WE[210_WIND_2]"
"r_WE[211_WIND_2]"
"r_WE[212_WIND_2]"
"r_WE[213_WIND_2]"
"r_WE[214_WIND_2]"
"r_WE[215_WIND_2]"
"r_WE[216_WIND_2]"
"r_WE[217_WIND_2]"
"r_WE[218_WIND_2]"
"r_WE[219_WIND_2]"
"r_WE[220_WIND_2]"
"r_WE[221_WIND_2]"
"r_WE[222_WIND_2]"
"r_WE[223_WIND_2]"
"r_WE[224_WIND_2]"
"r_WE[301_WIND_2]"
"r_WE[302_WIND_2]"
"r_WE[303_WIND_1]"
"r_WE[303_WIND_2]"
"r_WE[304_WIND_2]"
"r_WE[305_WIND_2]"
"r_WE[306_WIND_2]"
"r_WE[307_WIND_2]"
"r_WE[308_WIND_2]"
"r_WE[309_WIND_1]"
"r_WE[309_WIND_2]"
"r_WE[310_WIND_2]"
"r_WE[311_WIND_2]"
"r_WE[312_WIND_2]"
"r_WE[313_WIND_2]"
"r_WE[314_WIND_2]"
"r_WE[315_WIND_2]"
"r_WE[316_WIND_2]"
"r_WE[317_WIND_1]"
"r_WE[317_WIND_2]"
"r_WE[318_WIND_2]"
"r_WE[319_WIND_2]"
"r_WE[320_WIND_2]"
"r_WE[321_WIND_2]"
"r_WE[322_WIND_2]"
"r_WE[323_WIND_2]"
"r_WE[324_WIND_2]"
"r_WE[325_WIND_2]"
"m_EE[101_STORAGE_1]"
"m_EE[102_STORAGE_1]"
"m_EE[103_STORAGE_1]"
"m_EE[104_STORAGE_1]"
"m_EE[105_STORAGE_1]"
"m_EE[106_STORAGE_1]"
"m_EE[107_STORAGE_1]"
"m_EE[108_STORAGE_1]"
"m_EE[109_STORAGE_1]"
"m_EE[110_STORAGE_1]"
"m_EE[111_STORAGE_1]"
"m_EE[112_STORAGE_1]"
"m_EE[113_STORAGE_1]"
"m_EE[114_STORAGE_1]"
"m_EE[115_STORAGE_1]"
"m_EE[116_STORAGE_1]"
"m_EE[117_STORAGE_1]"
"m_EE[118_STORAGE_1]"
"m_EE[119_STORAGE_1]"
"m_EE[120_STORAGE_1]"
"m_EE[121_STORAGE_1]"
"m_EE[122_STORAGE_1]"
"m_EE[123_STORAGE_1]"
"m_EE[124_STORAGE_1]"
"m_EE[201_STORAGE_1]"
"m_EE[202_STORAGE_1]"
"m_EE[203_STORAGE_1]"
"m_EE[204_STORAGE_1]"
"m_EE[205_STORAGE_1]"
"m_EE[206_STORAGE_1]"
"m_EE[207_STORAGE_1]"
"m_EE[208_STORAGE_1]"
"m_EE[209_STORAGE_1]"
"m_EE[210_STORAGE_1]"
"m_EE[211_STORAGE_1]"
"m_EE[212_STORAGE_1]"
"m_EE[213_STORAGE_1]"
"m_EE[214_STORAGE_1]"
"m_EE[215_STORAGE_1]"
"m_EE[216_STORAGE_1]"
"m_EE[217_STORAGE_1]"
"m_EE[218_STORAGE_1]"
"m_EE[219_STORAGE_1]"
"m_EE[220_STORAGE_1]"
"m_EE[221_STORAGE_1]"
"m_EE[222_STORAGE_1]"
"m_EE[223_STORAGE_1]"
"m_EE[224_STORAGE_1]"
"m_EE[301_STORAGE_1]"
"m_EE[302_STORAGE_1]"
"m_EE[303_STORAGE_1]"
"m_EE[304_STORAGE_1]"
"m_EE[305_STORAGE_1]"
"m_EE[306_STORAGE_1]"
"m_EE[307_STORAGE_1]"
"m_EE[308_STORAGE_1]"
"m_EE[309_STORAGE_1]"
"m_EE[310_STORAGE_1]"
"m_EE[311_STORAGE_1]"
"m_EE[312_STORAGE_1]"
"m_EE[313_STORAGE_1]"
"m_EE[314_STORAGE_1]"
"m_EE[315_STORAGE_1]"
"m_EE[316_STORAGE_1]"
"m_EE[317_STORAGE_1]"
"m_EE[318_STORAGE_1]"
"m_EE[319_STORAGE_1]"
"m_EE[320_STORAGE_1]"
"m_EE[321_STORAGE_1]"
"m_EE[322_STORAGE_1]"
"m_EE[323_STORAGE_1]"
"m_EE[324_STORAGE_1]"
"m_EE[325_STORAGE_1]"
];

    fs = Model(with_optimizer(Gurobi.Optimizer, OutputFlag = 0));
    #set_optimizer_attribute(fs, "msg_lev", GLPK.GLP_MSG_ALL)

    x = Vector{VariableRef}()
    for i = 1:150
        push!(x, @variable(fs, base_name = names[i], lower_bound = 0.0))
    end
    
    #@objective(fs, Min, sum(0.9040815000000001*x[i] for i = 1:77)+sum(1.1059811266666666*x[i] for i = 78:150))
    @objective(fs, Min, 0.0)
        
    return fs
    
end

create_first_stage (generic function with 1 method)

### Run algorithm

In [12]:
xn, firststage, fs = LShaped.L_Shaped_Algorithm(subproblem_uploader, 
                                        v_dict, 2, create_first_stage, 1e-6, 250)#, [0.4, 0.6]; store="./bl_data/");

Loading in variable dataframe...
Loading in constraint dataframe...
Loading in constraint matrix...
Creating model...
Academic license - for non-commercial use only - expires 2022-08-19
Creating variable 1000...
Creating variable 2000...
Creating variable 3000...
Creating variable 4000...
Creating variable 5000...
Creating variable 6000...
Creating variable 7000...
Creating variable 8000...
Creating variable 9000...
Creating variable 10000...
Creating variable 11000...
Creating variable 12000...
Creating variable 13000...
Creating variable 14000...
Creating variable 15000...
Creating variable 16000...
Creating variable 17000...
Creating variable 18000...
Creating variable 19000...
Creating variable 20000...
Creating variable 21000...
Creating variable 22000...
Creating variable 23000...
Creating variable 24000...
Creating variable 25000...
Creating variable 26000...
Creating variable 27000...
Creating variable 28000...
Creating variable 29000...
Creating variable 30000...
Creating vari

Creating Constraint 29000...
For this iteration: nzt =7.1e-6, expt = 6.87e-5, cont = 1.04e-5, tot = 8.62e-5
Creating Constraint 30000...
For this iteration: nzt =4.1e-6, expt = 4.07e-5, cont = 5.9e-6, tot = 5.07e-5
Creating Constraint 31000...
For this iteration: nzt =5.0e-6, expt = 5.09e-5, cont = 6.8e-6, tot = 6.27e-5
Creating Constraint 32000...
For this iteration: nzt =6.8e-6, expt = 7.02e-5, cont = 8.5e-6, tot = 8.55e-5
Creating Constraint 33000...
For this iteration: nzt =4.7e-6, expt = 4.79e-5, cont = 5.8e-6, tot = 5.84e-5
Creating Constraint 34000...
For this iteration: nzt =4.2e-6, expt = 5.61e-5, cont = 7.2e-6, tot = 6.75e-5
Creating Constraint 35000...
For this iteration: nzt =6.4e-6, expt = 7.05e-5, cont = 2.34e-5, tot = 0.0001003
Creating Constraint 36000...
For this iteration: nzt =3.5e-6, expt = 3.68e-5, cont = 4.3e-6, tot = 4.46e-5
Creating Constraint 37000...
For this iteration: nzt =4.7e-6, expt = 6.15e-5, cont = 6.3e-6, tot = 7.25e-5
Creating Constraint 38000...
For 

For this iteration: nzt =2.7e-6, expt = 1.29e-5, cont = 5.9e-6, tot = 2.15e-5
Creating Constraint 104000...
For this iteration: nzt =3.1e-6, expt = 1.49e-5, cont = 7.0e-6, tot = 2.5e-5
Creating Constraint 105000...
For this iteration: nzt =2.3e-6, expt = 1.25e-5, cont = 5.8e-6, tot = 2.06e-5
Creating Constraint 106000...
For this iteration: nzt =3.0e-6, expt = 1.55e-5, cont = 5.9e-6, tot = 2.44e-5
Creating Constraint 107000...
For this iteration: nzt =3.5e-6, expt = 1.89e-5, cont = 5.8e-6, tot = 2.82e-5
Creating Constraint 108000...
For this iteration: nzt =2.9e-6, expt = 2.15e-5, cont = 7.5e-6, tot = 3.19e-5
Creating Constraint 109000...
For this iteration: nzt =2.9e-6, expt = 1.99e-5, cont = 6.8e-6, tot = 2.96e-5
Creating Constraint 110000...
For this iteration: nzt =3.7e-6, expt = 2.14e-5, cont = 7.6e-6, tot = 3.27e-5
Creating Constraint 111000...
For this iteration: nzt =4.8e-6, expt = 3.18e-5, cont = 1.0e-5, tot = 4.66e-5
Creating Constraint 112000...
For this iteration: nzt =2.0e

For this iteration: nzt =3.5e-6, expt = 2.03e-5, cont = 9.6e-6, tot = 3.34e-5
Creating Constraint 185000...
For this iteration: nzt =4.6e-6, expt = 2.85e-5, cont = 1.46e-5, tot = 4.77e-5
Creating Constraint 186000...
For this iteration: nzt =3.5e-6, expt = 5.54e-5, cont = 9.3e-6, tot = 6.82e-5
Creating Constraint 187000...
For this iteration: nzt =2.6e-6, expt = 1.5e-5, cont = 7.2e-6, tot = 2.48e-5
Creating Constraint 188000...
For this iteration: nzt =3.1e-6, expt = 1.8e-5, cont = 8.5e-6, tot = 2.96e-5
Creating Constraint 189000...
For this iteration: nzt =3.5e-6, expt = 2.11e-5, cont = 1.0e-5, tot = 3.46e-5
Creating Constraint 190000...
For this iteration: nzt =3.1e-6, expt = 2.56e-5, cont = 9.3e-6, tot = 3.8e-5
Creating Constraint 191000...
For this iteration: nzt =3.2e-6, expt = 1.73e-5, cont = 8.7e-6, tot = 2.92e-5
Creating Constraint 192000...
For this iteration: nzt =4.4e-6, expt = 1.97e-5, cont = 1.46e-5, tot = 3.87e-5
Creating Constraint 193000...
For this iteration: nzt =2.2e

Creating Constraint 264000...
For this iteration: nzt =5.3e-6, expt = 2.81e-5, cont = 1.29e-5, tot = 4.63e-5
Creating Constraint 265000...
For this iteration: nzt =6.4e-6, expt = 3.67e-5, cont = 1.46e-5, tot = 5.77e-5
Creating Constraint 266000...
For this iteration: nzt =9.3e-6, expt = 4.54e-5, cont = 3.0e-5, tot = 8.47e-5
Creating Constraint 267000...
For this iteration: nzt =6.4e-6, expt = 3.11e-5, cont = 2.14e-5, tot = 5.89e-5
Creating Constraint 268000...
For this iteration: nzt =4.8e-6, expt = 4.75e-5, cont = 2.04e-5, tot = 7.27e-5
Creating Constraint 269000...
For this iteration: nzt =1.35e-5, expt = 5.12e-5, cont = 1.55e-5, tot = 8.02e-5
Creating Constraint 270000...
For this iteration: nzt =5.2e-6, expt = 4.03e-5, cont = 1.45e-5, tot = 6.0e-5
Creating Constraint 271000...
For this iteration: nzt =3.8e-6, expt = 2.75e-5, cont = 1.05e-5, tot = 4.18e-5
Creating Constraint 272000...
For this iteration: nzt =4.9e-6, expt = 4.32e-5, cont = 1.3e-5, tot = 6.11e-5
Creating Constraint 2

...

Creating Constraint 42000...
For this iteration: nzt =6.0e-6, expt = 4.8e-5, cont = 1.01e-5, tot = 6.41e-5
Creating Constraint 43000...
For this iteration: nzt =0.0001542, expt = 0.0022663, cont = 0.0001372, tot = 0.0025577
Creating Constraint 44000...
For this iteration: nzt =0.00017, expt = 0.0026528, cont = 4.92e-5, tot = 0.002872
Creating Constraint 45000...
For this iteration: nzt =0.0001223, expt = 0.0022591, cont = 5.19e-5, tot = 0.0024333
Creating Constraint 46000...
For this iteration: nzt =0.0001484, expt = 0.0022234, cont = 5.09e-5, tot = 0.0024227
Creating Constraint 47000...
For this iteration: nzt =0.000131, expt = 0.0020749, cont = 4.33e-5, tot = 0.0022492
Creating Constraint 48000...
For this iteration: nzt =0.0001989, expt = 0.0027679, cont = 7.68e-5, tot = 0.0030436
Creating Constraint 49000...
For this iteration: nzt =0.0001504, expt = 0.0020486, cont = 4.67e-5, tot = 0.0022457
Creating Constraint 50000...
For this iteration: nzt =0.0001724, expt = 0.0023333, cont = 

For this iteration: nzt =1.0e-6, expt = 6.9e-6, cont = 2.8e-6, tot = 1.07e-5
Creating Constraint 205000...
For this iteration: nzt =2.7e-6, expt = 1.16e-5, cont = 7.5e-6, tot = 2.18e-5
Creating Constraint 206000...
For this iteration: nzt =2.4e-6, expt = 1.35e-5, cont = 8.1e-6, tot = 2.4e-5
Creating Constraint 207000...
For this iteration: nzt =1.1e-6, expt = 6.8e-6, cont = 4.2e-6, tot = 1.21e-5
Creating Constraint 208000...
For this iteration: nzt =1.3e-6, expt = 9.3e-6, cont = 4.2e-6, tot = 1.48e-5
Creating Constraint 209000...
For this iteration: nzt =1.2e-6, expt = 6.8e-6, cont = 3.0e-6, tot = 1.1e-5
Creating Constraint 210000...
For this iteration: nzt =2.2e-6, expt = 1.0e-5, cont = 7.5e-6, tot = 1.97e-5
Creating Constraint 211000...
For this iteration: nzt =9.0e-6, expt = 9.9e-6, cont = 6.0e-6, tot = 2.49e-5
Creating Constraint 212000...
For this iteration: nzt =2.2e-6, expt = 1.05e-5, cont = 6.2e-6, tot = 1.89e-5
Creating Constraint 213000...
For this iteration: nzt =1.5e-6, exp

For this iteration: nzt =3.5e-6, expt = 2.62e-5, cont = 2.05e-5, tot = 5.02e-5
Creating Constraint 287000...
For this iteration: nzt =4.3e-6, expt = 2.81e-5, cont = 1.17e-5, tot = 4.41e-5
Creating Constraint 288000...
For this iteration: nzt =3.0e-6, expt = 2.03e-5, cont = 9.1e-6, tot = 3.24e-5
Creating Constraint 289000...
For this iteration: nzt =2.7e-6, expt = 1.88e-5, cont = 7.4e-6, tot = 2.89e-5
Creating Constraint 290000...
For this iteration: nzt =2.7e-6, expt = 1.78e-5, cont = 1.36e-5, tot = 3.41e-5
Creating Constraint 291000...
For this iteration: nzt =2.6e-6, expt = 1.66e-5, cont = 6.7e-6, tot = 2.59e-5
Creating Constraint 292000...
For this iteration: nzt =2.9e-6, expt = 1.85e-5, cont = 7.9e-6, tot = 2.93e-5
Creating Constraint 293000...
For this iteration: nzt =2.9e-6, expt = 2.72e-5, cont = 1.24e-5, tot = 4.25e-5
Creating Constraint 294000...
For this iteration: nzt =3.2e-6, expt = 2.0e-5, cont = 9.3e-6, tot = 3.25e-5
Creating Constraint 295000...
For this iteration: nzt =

For this iteration: nzt =2.9e-6, expt = 1.4e-5, cont = 5.13e-5, tot = 6.82e-5
Creating Constraint 367000...
For this iteration: nzt =3.2e-6, expt = 4.82e-5, cont = 1.56e-5, tot = 6.7e-5
Creating Constraint 368000...
For this iteration: nzt =6.5e-6, expt = 2.91e-5, cont = 2.2e-5, tot = 5.76e-5
Creating Constraint 369000...
For this iteration: nzt =2.4e-6, expt = 1.45e-5, cont = 9.1e-6, tot = 2.6e-5
Creating Constraint 370000...
For this iteration: nzt =2.1e-6, expt = 1.51e-5, cont = 1.11e-5, tot = 2.83e-5
Creating Constraint 371000...
For this iteration: nzt =1.9e-6, expt = 8.8e-6, cont = 6.5e-6, tot = 1.72e-5
Creating Constraint 372000...
For this iteration: nzt =2.2e-6, expt = 1.02e-5, cont = 7.5e-6, tot = 1.99e-5
Creating Constraint 373000...
For this iteration: nzt =2.4e-6, expt = 9.5e-6, cont = 7.2e-6, tot = 1.91e-5
Creating Constraint 374000...
For this iteration: nzt =6.5e-6, expt = 2.3e-5, cont = 2.11e-5, tot = 5.06e-5
Creating Constraint 375000...
For this iteration: nzt =2.9e-

LoadError: BoundsError: attempt to access 2×155447 Matrix{Float64} at index [2, 345455]

In [ ]:
rm("bl_data",recursive=true)

In [1]:
grad1 = [0.5037776984815919, 0.06373730398486477, 0.41443672560252487, 0.6290541488266383, 0.48831557240989726, 0.3198977546379106, -1.4701274879558595, -0.3572346407095045, 0.42977830932732525, 0.28000570210758813, -0.2853183248364431, 0.03116546210243437, 0.015692930609591482, 0.46877694233611905, 0.7933613227504624, 0.1294532112803345, 0.7191691109690904, 0.5426707348899168, -0.5613043958565942, -0.4731712249771425, 0.5631003877710028, 0.9030626179589812, 0.8041986310157428, -0.5690404494787102, -0.5365164032912857, -0.906934064141254, -0.28535166597376843, 0.11918265614270779, -0.6464609305128861, -0.0760894870392593, -0.7307494526621531, -0.42180809182317647, 0.03167776946143619, 0.11896701275567084, -0.8544374061286103, -0.5215030674401115, 0.3739388622307878, 0.3783515051963955, 0.22948825168131837, 0.39675715844333975, -1.9149866156801298, -0.37389947640005217, -0.6298634236084494, -0.344408530540614, 0.39600416770574615, -0.9844347584256298, 0.6095438630711323, -0.673895058652492, 0.5856530380142381, 0.41789417891366926, 0.1910441579654843, -9.524072591614466, 0.3173944712306791, 0.43505263279910705, -0.35271884836723993, -0.8053699575572271, 0.7499932485771639, 0.35959516355770954, -5.458141687449131, 0.1254512509538649, 0.7078748393871295, 0.7640471826140545, 0.5993252158202006, 0.7479764949362304, 0.6911978291020666, 0.23432377556755263, 0.121428543110781, -4.094830033780942, 0.17437353425241792, 0.532151767848811, 0.5897197731453963, -0.10017964619554548, 0.2539023765108499, 0.6921202103698108, 0.40492160378704467, -0.46916782746548324, -0.36526598967218915, -3.522233344326458, -3.5239382243081856, -3.299074942342414, -3.5492679299848393, -3.5691080503015025, -3.5963431825400574, -3.4637095687921726, -3.518082812179138, -3.5656036332328833, -3.6176371576639026, -3.717965622913785, -3.645531919028192, -3.642273326090837, -3.907581402832581, -2.2535778832176936, -4.173946427004225, 0.45695685161572064, 0.3765312755834419, -3.9990601837951067, -3.853513360492696, -0.7718909615161245, 1.0840779041409179, -3.773462607676371, -3.0179420580328484, -3.203890071096273, -3.2041189590102372, -3.3236927133542995, -3.2200865335730375, -3.1835152442703434, -3.173093100389966, -3.1785410445618822, -3.1785410445618822, -3.2396324021879366, -3.1667025444155725, -2.555255961360167, -3.201224147152419, -3.1698134623871335, -2.6070749266973534, -3.4004194170531696, -3.349218889757215, -3.4441222547787076, -3.4344292184050063, -3.2215902803298406, -2.4789193040283934, -3.425428541772285, -3.4327763596014003, -2.4275137635096278, -3.373382380185074, -1.843563896104754, -1.8466355900492983, -1.924459866785347, -1.837699482577906, -1.83447387233678, -1.3895059096243072, -1.8242629399058137, -1.8242629399058132, -1.3998782033705188, -1.819224924483763, -1.838688712278553, -1.79120956343142, -1.7841110783529412, -1.9038823911150378, -2.0427945389143174, -1.99546398757581, -2.119020424371989, -2.184299335966259, -1.8578299112219487, -1.7381481056968475, -2.1344851100020232, -2.128406505245711, -1.6723231126580411, -1.9915427916212867, -1.5958713262403288]

grad2 = [0.39244073882461405, -0.12012372293680723, 0.001721744289706173, 0.49210278005806285, 0.3740212025821536, 0.1867955695865629, -1.8967733161112017, -0.5895078484031353, 0.3230064274832319, 0.1701116477428899, -0.43652925621981187, -0.08818362195193247, -0.15158610206727746, 0.3661082956749621, 0.37095045747249755, 0.09612761172960113, -0.5495621987273913, -1.0303292677354716, -0.6803503874306378, -0.6177736682906209, -0.1357747409166341, -23.105432678371592, -0.19036560522299437, -0.7464074669004639, -1.3445433256759904, -1.194464557248188, -0.44962780110647915, -0.03443700647967568, -0.8791761299201539, -0.15941637344821194, -0.9479044138620388, -0.5819311916998002, -0.02208895810601666, -0.01370222499340834, -1.0527123906549387, -0.8658443667066091, -0.10932948949668753, 0.14400044693089853, 0.055402002154857355, 0.271062889096674, -2.475688944649784, -0.6575682581814308, -0.9711837951244174, -0.7050997307288138, 0.13238624385237588, -1.3903558167666414, 0.531988598248941, -1.1913840162105238, 0.48036947762068105, -0.021377158275123487, -0.4398093347362492, -17.607049410816106, -0.19240509151393004, -0.24976018781690434, -1.4095040392353058, -2.2402674350458196, 0.3287750830052245, -0.31564938690649647, -11.004605466476471, -0.774257866683844, 0.45331445283420063, 0.35102550926951015, 0.07640486478318598, 0.4111056822147123, 0.3910500997947332, -0.3407597373614967, -0.5628533462561573, -7.668235043005394, -0.4213447157735705, 0.1663141576985101, 0.25602110227544833, -1.1440627312365783, -0.35832186059244076, 0.47454333562382156, -0.11789307791074788, -1.5622209484676928, -1.7591536239641883, -8.998893832182649, -8.998882393699217, -8.999251963058652, -8.998849332626847, -8.998819009385738, -8.998775505162946, -8.996735285945052, -8.997605276210214, -8.998822258992778, -8.998741545784235, -8.999032944673038, -8.998914771516217, -8.998939440638978, -8.99936058155556, -8.99998057500011, -8.999820833366723, -9.000094772336706, -9.000222364680031, -8.999541801903602, -8.999299165848711, -9.00034084328455, -9.000244121755916, -8.999165716018526, -8.999704545414245, -8.99867690757352, -8.998656792822617, -8.999310025713262, -8.998600028831232, -8.998543985255038, -8.998466749809552, -8.99847995824669, -8.99847995824669, -8.998553544932776, -8.99840637156061, -8.998352838123138, -8.998204160870593, -8.998151153049795, -8.998405510920724, -8.998636821107626, -8.998479503660194, -8.99865186892906, -8.998648060985948, -8.998143030066425, -8.99785044433271, -8.998644525038774, -8.998647411654607, -8.997689522179172, -8.998891860008202, -8.993098873423468, -8.998517414164136, -8.875966579404448, -9.035793980397987, -9.007386585576082, -9.016473706281012, -9.044249121177156, -9.044249121177156, -9.066319672432, -9.02217856992231, -9.025884406814399, -9.029373408983595, -9.023420842269575, -9.010236791488657, -8.981179826889502, -8.988255617578682, -8.9798040246775, -8.995064464193877, -8.996736223060854, -9.004110662610573, -8.989762230975611, -8.985848022719706, -9.008166604362918, -8.941320517045247, -9.00793943588342]


150-element Vector{Float64}:
  0.39244073882461405
 -0.12012372293680723
  0.001721744289706173
  0.49210278005806285
  0.3740212025821536
  0.1867955695865629
 -1.8967733161112017
 -0.5895078484031353
  0.3230064274832319
  0.1701116477428899
 -0.43652925621981187
 -0.08818362195193247
 -0.15158610206727746
  ⋮
 -9.010236791488657
 -8.981179826889502
 -8.988255617578682
 -8.9798040246775
 -8.995064464193877
 -8.996736223060854
 -9.004110662610573
 -8.989762230975611
 -8.985848022719706
 -9.008166604362918
 -8.941320517045247
 -9.00793943588342

In [2]:
grad1 - grad2

150-element Vector{Float64}:
 0.11133695965697787
 0.183861026921672
 0.4127149813128187
 0.1369513687685755
 0.11429436982774366
 0.1331021850513477
 0.42664582815534224
 0.2322732076936308
 0.10677188184409336
 0.10989405436469823
 0.15121093138336877
 0.11934908405436684
 0.16727903267686894
 ⋮
 7.106354400373619
 6.938385287975184
 6.992791630002872
 6.860783600305512
 6.810765128227619
 7.138906311838905
 7.265962556913726
 6.855277120973588
 6.857441517473995
 7.3358434917048765
 6.94977772542396
 7.412068109643092

In [4]:
798+1547+1109+579+150

4183